# Clean extracted data
To do:
* Find Names
* Create function getAllCandidats

In [8]:
import json
import pandas as pd
from pathlib import Path
import re
import utils

In [9]:
# Get all files
df = pd.read_csv(Path('../export/dokumente.csv'))

# Only "Protokoll"
df = df[df.dokument_kategorie == 'Protokoll']

df_raete = pd.read_csv(Path('../export/mitglieder.csv'))
df_raete['_name'] = df_raete['vorname'] +' ' + df_raete['name']
df_raete['_name'] = df_raete['_name'].str.lower()


# Some names are written wrong. Some changed their name (marriage). Add
# Aliases to the dataframe

for a in utils.aliases:
    df_raete = df_raete.append({
        '_name': a[0].lower(),
        'name': a[2],
        'vorname': a[1]}, ignore_index=True)

In [10]:
# Remove this
remove = ['Ratsvizepräsidentin', 'Ratspräsidentin', 'Regierungsrätin',
     'Ratspräsident', 'Regierungsrat', 'Ratsvizepräsident', 
    ', Niederhasli.', ', SVP, Zürich.', ', SVP, Dinhard.',
    ', GLP, Dübendorf.', ', SVP, Volketswil.', ', GLP, Dübendorf.',
    ', SVP, Uetikon am See.']

records = []
notfound = []

# Loop file
for i, row in df.iterrows():

    # Create Name and load
    f = Path('../export/extracts/%s' % row['_filename']).with_suffix('.json')
    data = json.load(open(f))

    filename = Path(f).stem

    for msg in data:
        #print(msg)

        # Replace double space
        msg = msg.replace('  ', ' ')

        if msg.strip() == '':
            continue

        # Remove Strings
        for r in remove:
            msg = msg.replace(r, '')

        msg = msg.strip()

        # Extract Name
        r1 = re.findall(r"[^(:]*", msg)
        if len(r1) > 0:
            name = r1[0].strip()
            r = df_raete[df_raete['_name'] == name.lower()]
            # If name not found, add it to a list for manual check
            if len(r) == 0:
                notfound.append({
                    'name': name,
                    'f': filename,
                })
                
            else:
                # Name found!

                # Clean text
                text = msg
                text = text.replace('-\n', '')
                text = text.replace('\n', ' ')
                text = text.replace('  ', ' ')
                text = text.strip()

                text = re.findall(r".*:(.*)", text)
                if len(text) > 0:
                    text = text[0].strip()

                    records.append({
                        'name': r.iloc[0]['name'],
                        'vorname': r.iloc[0]['vorname'],
                        'text': text,
                        'f': filename
                    })
        else:
            # No Name found
            print('!! No Name found in:\n%s' % msg)

print("finito")

finito


## Join with dataframe and save

In [11]:
df_votum = pd.DataFrame(records)
df_votum.to_csv(Path('../export/df_votum.csv'), index=False)